In [2]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os

from reranking import data_prep

# Step 1 : Prep input for tfrecord with expansions (optional)

This step is optional in case the query at Retrieval is the same for Reranking

In [ ]:
def prep_input_dl4marco_bert(queryFile, retrievedFile, outputFile):
    """
    Takes the query file with the chosen expansion and replaces the query from Step 1 with the expansion.

    Input:
    - queryFile : [qid, query] (either query, utterance or expansion)
    - retrievedFile: [qID, passageID, queryStep1, passage]

    Output: [query_id   passage_id  expanded_query   passage]
    """
    queryDF = pd.read_csv(queryFile, delimiter="\t", header=None)
    assert len(queryDF.columns) == 2
    queryDict = dict(zip(queryDF[0], queryDF[1]))

    dataframe = pd.read_csv(retrievedFile, delimiter="\t", header=0)
    assert len(dataframe.columns) == 4

    # new column
    dataframe["expanded_query"] = dataframe['query_id'].map(queryDict)

    columnsTitles = ["query_id", "passage_id", "expanded_query", "passage"]

    dataframe = dataframe.reindex(columns=columnsTitles)

    dataframe.to_csv(outputFile, sep='\t', index=False, header=False)

### E.g. BERT-BERTContext

In [2]:
def prepInputBERT(i):

    query_path = "../data/rewritten-utterances-classif/"
    queryFile = query_path + "classif_results_strategy"+str(i)+"_S1_BERT_S2_BERT_Context.tsv"

    path = "../data/reranking_BERT_input/"
    basename_file = "S1_BERT_S2_BERT_Context_strategy"+str(i)+"_default_NOSW_PRF20"

    retrievedFile = path+basename_file + "_withQueryDocs.txt"
    outputFile = path+basename_file+"_bert4msmarco.txt"
    prep_input_dl4marco_bert(queryFile, retrievedFile, outputFile)

In [ ]:
# 5 strategies
for i in [1,2,3,4,5]:
    prepInputBERT(i)

### E.g. Ground Truth

In [17]:
def prepInputBERT_GT(i):

    query_path = "../data/rewritten-utterances-classif/"
    queryFile = query_path + "classif_results_GT_strategy"+str(i)+".tsv"

    path = "../data/reranking_BERT_input/"
    basename_file = "GT_strategy"+str(i)+"_default_NOSW_PRF20"

    retrievedFile = path+basename_file + "_withQueryDocs.txt"
    outputFile = path+basename_file+"_bert4msmarco.txt"
    prep_input_dl4marco_bert(queryFile, retrievedFile, outputFile)

In [ ]:
# 5 strategies Ground Truth
for i in [1,2,3,4,5]:
    prepInputBERT_GT(i)

Switch tp repo: **dl4marco-bert**

# Step 2: Convert input to TFrecord

```
bash convert_CAST_to_tfrecord_onefile.sh
```

# Step 3: Run model in inference

```
bash test_CAST.sh
```

# Step 4: Evaluate

    using **trec_eval**